In [ ]:
import sys
sys.path.append("../..")
from IPython.display import display
%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
import networkx as nx
import magine.ontology.enrichment_tools as et
import magine.networks.visualization.notebook_tools as nt
import magine.networks.visualization.notebooks.view as view
from magine.networks.network_subgraphs import NetworkSubgraphs
from exp_data import exp_data
from magine.networks.visualization.igraph_tools import paint_network_overtime
from magine.plotting.heatmaps import heatmap_by_terms

# Exploring network using enrichment analysis

### This example uses the enrichment output we just obtained to explore the network.

First, lets load the molecular network and the enrichment output from the previous runs. 

In [ ]:
enrichment_array = pd.read_csv('Data/cisplatin_enrichment.csv.gz', index_col=0)
network = nx.read_gpickle('Data/cisplatin_based_network.p')
subgraph_gen = NetworkSubgraphs(network=network, exp_data=exp_data)

For this example, we will only look at the proteomics data. 

In [ ]:
proteomics = et.filter_rows(enrichment_array, column='category', options=['proteomics_up'])
proteomics = proteomics[~proteomics['term_name'].isnull()]

In [ ]:
print_cols = ['term_name', 'combined_score', 'adj_p_value', 'n_genes', 'rank']

In [ ]:
display(proteomics.head(10))
display(proteomics[print_cols].head(10))

For this example, we just want to look at "biological processes" descriptions, so we will limit our analysis to only databases with this type of information

In [ ]:
print(proteomics['db'].unique())

In [ ]:
process_dbs = [
        'GO_Biological_Process_2017',
        'Humancyc_2016',
        'Reactome_2016',
        'KEGG_2016',
        'BioCarta_2016',
        'Humancyc_2016',
        'NCI-Nature_2016',
        'Panther_2016',
        'WikiPathways_2016',
]

In [ ]:
time_1_hour_prot = et.filter_dataframe(proteomics, 
                                       p_value=0.05,
                                       combined_score=0.0, 
                                       db=process_dbs,
                                      )
display(time_1_hour_prot[print_cols].head(15))

In [ ]:
time_1_hour_prot = et.filter_dataframe(proteomics, 
                                       p_value=0.05,
                                       combined_score=0.0, 
                                       sample_id='01hr', 
                                       db=process_dbs,
                                      )
display(time_1_hour_prot[print_cols].head(15))

Since terms across databases might be redundant ("Interleukin-3, 5 and GM-CSF signaling_Homo sapiens_R-HSA-512988" and "Interleukin receptor SHC signaling_Homo sapiens_R-HSA-912526" have nearly a full overlap of genes), we want to eliminate duplicate terms and focus on the most enriched. 

For that we use the Jaccard Index. It is impelmented in the find_similar_terms function in enrichment_tools (et).

In [ ]:
filtered_1hr = et.filter_similar_terms(time_1_hour_prot, threshold=.7)
display(filtered_1hr[print_cols].head(20))

Now we can explore the top hits, which has been slimmed from 89 to 33 terms. Generally this is where the expert knowledge comes in. However, a quick search with each term and search terms of you molecule of interest tend to be useful. 

The first hit is 'negative regulation of transcription', which means that something caused genes not to be transcribed. Cisplatin causes DNA damage, thus negative regulaton of transcription makes sense. So does top hit 2, 'Cell Cycle_Homo sapiens_hsa'. 

A quick search for 'Interleukin-2 signaling' and 'Cisplatin' __[link](https://www.google.com/search?rlz=1C1CHBD_enUS721US721&ei=KzNeWuCxBsfq_AaSgYuYDQ&q=Interleukin-2+signaling+cisplatin&oq=Interleukin-2+signaling+cisplatin&gs_l=psy-ab.3..35i39k1.8097.9052.0.9196.10.10.0.0.0.0.145.897.7j3.10.0....0...1c.1.64.psy-ab..3.2.218....0.TInUjcZY740)__ returns a paper titled "Cisplatin at clinically relevant concentrations enhances interleukin-2 synthesis by human primary blood lymphocytes." __[link](https://www.ncbi.nlm.nih.gov/pubmed/10211553)__


We can link the two together to see how once might regulate the other by looking at the molecular interactions.

## Expected findings

In [ ]:
shorten_names = {
    'Cell Cycle_hsa':'Cell Cycle',
    'DNA Repair_hsa' : 'DNA Repair',
                }
rename_all = proteomics.copy()
rename_all['term_name'] = rename_all['term_name'].replace(shorten_names)
dna_and_cellcycle_term, dna_and_cellcycle_mole = nt.create_subnetwork(['Cell Cycle', 'DNA Repair'], 
                                                                      rename_all, 
                                                                      network, 
                                                                      'entire_network')

In [ ]:
view.render_graph(dna_and_cellcycle_mole, add_parent=True)

In [ ]:
view.render_graph(dna_and_cellcycle_term)

In [ ]:
rename_all['term_name'] = rename_all['term_name'].replace(shorten_names)
['Cell Cycle', 'DNA Repair']

In [ ]:
paint_network_overtime(dna_and_cellcycle_mole, exp_data, 'red', 'dna_cell_cycle' )

In [ ]:
view.display_graph(dna_and_cellcycle_term)

In [ ]:
view.display_graph(dna_and_cellcycle_mole, add_parent=True)

In [ ]:
# dna_genes = et.term_to_genes(df=proteomics, term='DNA Repair_Homo sapiens_R-HSA-73894')
dna_genes = et.term_to_genes(df=time_1_hour_prot, term='DNA Repair_hsa')
print(len(dna_genes))

In [ ]:
dna_network = subgraph_gen.shortest_paths_between_lists(
    dna_genes, max_length=3
)
print(len(dna_network.nodes()))

In [ ]:
print(len(dna_network.nodes()))

In [ ]:
view.render_graph(dna_network)

In [ ]:

plus_neighbors = subgraph_gen.expand_neighbors(dna_network, down_stream=True, 
                                               expand_all=True,
                                               include_list=exp_data.sig_species_over_time['06hr'])

In [ ]:
print(len(plus_neighbors.nodes()))
print(len(plus_neighbors.edges()))

In [ ]:
view.render_graph(plus_neighbors)

In [ ]:
# view.render_graph(plus_neighbors)
paint_network_overtime(plus_neighbors, exp_data, ['red', 'green', 'lightblue', 'purple'], 'dna' )

In [ ]:
# subgraph_gen.measured_networks_over_time(plus_neighbors, ['red', 'green', 'lightblue', 'purple'], 'dna')

In [ ]:
subgraph_gen.measured_networks_over_time(dna_network, ['red', 'green', 'lightblue', 'purple'], 'dna')

In [ ]:
from magine.plotting.species_plotting import plot_list_of_genes
plot_list_of_genes(exp_data.data, dna_genes, save_name='dna_genes', image_format='png', plot_type='matplotlib')

In [ ]:
time_1_hour_prot = et.filter_dataframe(proteomics, 
                                       p_value=0.05,
                                       combined_score=0.0, 
                                       sample_id='01hr', 
                                       db=process_dbs,
                                      )
kinases = et.filter_dataframe(proteomics, 
                              p_value=0.05, 
                              sample_id='01hr', 
                              combined_score=0.0, 
                              db='KEA_2015')
kinases = et.remove_redundant(kinases)

display(kinases[print_cols].head(20))

## Exploring other top hits from 1 hour

### Side effects of cisplatin
Chemotherapy-induced peripheral neuropathy. __[link](https://www.frontiersin.org/articles/10.3389/fnins.2017.00481/full)__
It is not well understood why cisplatin causes CIPN. Surprisely here, we see that Axon Guidance has a combined score of 20.5. 33 species are effected by cisplain that are linked with axon guidance. We are not neural experts and did not know that axon guidance was related to CIPN. Using MAGINE we were able to find ties between the two.

In [ ]:
axon_guidance = et.term_to_genes(df=renamed_1hr, term='Axon guidance_Homo sapiens_R-HSA-422475')
print(axon_guidance)

In [ ]:
g = subgraph_gen.neighbors_of_list(axon_guidance, up_stream=True, down_stream=False, max_dist=1, 
                                   include_only=exp_data.sig_species_over_time['01hr']
                                  )
nt.render_graph(g)

In [ ]:
print(chloride_transport)

In [ ]:
chloride_transport = et.term_to_genes(df=renamed_1hr, term='chloride transport')
print(chloride_transport)
chl_trans = subgraph_gen.neighbors_of_list(chloride_transport, max_dist=2, 
#                                            include_only=exp_data.list_species
                                          )
nt.display_graph(chl_trans)

# 6 hour time point

In [ ]:
time_6_hour_prot = et.filter_dataframe(proteomics, 
                                       p_value=0.05, 
                                       combined_score=0.0, 
                                       sample_id='06hr', 
                                       db=process_dbs
                                      )
slimmed = et.filter_similar_terms(time_6_hour_prot, threshold=.7)
display(slimmed[print_cols].head(25))

In [ ]:
shorten_names = {
    'protein sumoylation':'protein sumoylation',
    'Activation of the AP-1 family of transcription factors_Homo sapiens_R-HSA-450341' : 'AP1 activation',
    'response to cAMP' : 'response to cAMP',
                }
time_6_hour_prot['term_name'] = time_6_hour_prot['term_name'].replace(shorten_names)
term_net_6, mol_net_6 = nt.create_subnetwork(shorten_names.values(),
                                             time_6_hour_prot, 
                                             network,
                                             '06hr')


In [ ]:
view.display_graph(term_net_6)

In [ ]:
view.render_graph(mol_net_6)

In [ ]:
time_24_hour_prot = et.filter_dataframe(proteomics, p_value=0.05, combined_score=0.0, 
                                        category='proteomics_up', 
                                        sample_id='24hr', db=process_dbs)
slimmed = et.filter_similar_terms(time_24_hour_prot, threshold=.5)
display(slimmed[print_cols].head(25))

In [ ]:
shorten_names = {
                 'cellular response to DNA damage stimulus': 'DDR',
                 'negative regulation of apoptotic process': 'negative regulation of apoptosis',
                 'Apoptosis_Homo sapiens_R-HSA-109581' : 'Apoptosis',
                }
renamed = time_24_hour_prot.copy()
time_24_hour_prot['term_name'] = time_24_hour_prot['term_name'].replace(shorten_names)
term_net_24, mol_net_24 = nt.create_subnetwork(shorten_names.values(), time_24_hour_prot, network, '24hr', cytoscape_js=False)

In [ ]:
view.display_graph(term_net_24)

In [ ]:
view.display_graph(mol_net_24, add_parent=True )

In [ ]:
time_48_hour_prot = et.filter_dataframe(proteomics, p_value=0.05, combined_score=0.0, category='proteomics_up', 
                                        sample_id='48hr', db=process_dbs)
slimmed = et.filter_similar_terms(time_48_hour_prot, threshold=.5)
display(slimmed[print_cols].head(25))

In [ ]:
shorten_names = {
                 'membrane organization': 'Membrane Organization',
                 'negative regulation of apoptotic process': 'negative regulation of apoptosis',
                 'neutrophil degranulation' : 'neutrophil degranulation',
                }
time_48_hour_prot['term_name'] = time_48_hour_prot['term_name'].replace(shorten_names)
term_net_48, mol_net_48 = nt.create_subnetwork(shorten_names.values(), time_48_hour_prot, network, '48hr')

In [ ]:
view.display_graph(term_net_48)

In [ ]:
view.render_graph(mol_net_48, add_parent=True)